In [ ]:
"""
  The code here has been inspired by

  PySpark Cookbook:
  Over 60 recipes for implementing big data processing and analytics using
  Apache Spark and Python
"""

# Using Spark Dataframe

In [ ]:
!pip install pyspark


In [ ]:
import pyspark
print(pyspark.__version__)


3.5.4


In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("ColabPySpark") \
    .master("local[*]") \
    .getOrCreate()

print("Spark is running:", spark.version)


Spark is running: 3.5.4


In [ ]:
# # Get Spark context
sc = spark.sparkContext

print(f"Spark UI: {sc.uiWebUrl}")  # Check Spark UI link
print(f"Spark Version: {sc.version}")

Spark UI: http://9d17be5665ee:4040
Spark Version: 3.5.4


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
## Create flights RDD
flightsRDD = sc.textFile('/content/drive/My Drive/Pyspark/data/departure_delays.csv')\
.map(lambda line: line.split(",")).zipWithIndex()\
.filter(lambda row_idx: row_idx[1] > 0)\
.map(lambda row_idx: row_idx[0])

In [ ]:
flightsRDD.take(3)

[['01011245', '6', '602', 'ABE', 'ATL'],
 ['01020600', '-8', '369', 'ABE', 'DTW'],
 ['01021245', '-2', '602', 'ABE', 'ATL']]

In [ ]:
# Create flightsDF DataFrame
flightsDF = spark.read.options(header='true', inferSchema='true')\
    .csv('/content/drive/My Drive/Pyspark/data/departure_delays.csv')

flightsDF

DataFrame[date: int, delay: int, distance: int, origin: string, destination: string]

In [ ]:
flightsDF.collect()

[Row(date=1011245, delay=6, distance=602, origin='ABE', destination='ATL'),
 Row(date=1020600, delay=-8, distance=369, origin='ABE', destination='DTW'),
 Row(date=1021245, delay=-2, distance=602, origin='ABE', destination='ATL'),
 Row(date=1020605, delay=-4, distance=602, origin='ABE', destination='ATL'),
 Row(date=1031245, delay=-4, distance=602, origin='ABE', destination='ATL'),
 Row(date=1030605, delay=0, distance=602, origin='ABE', destination='ATL'),
 Row(date=1041243, delay=10, distance=602, origin='ABE', destination='ATL'),
 Row(date=1040605, delay=28, distance=602, origin='ABE', destination='ATL'),
 Row(date=1051245, delay=88, distance=602, origin='ABE', destination='ATL'),
 Row(date=1050605, delay=9, distance=602, origin='ABE', destination='ATL'),
 Row(date=1061215, delay=-6, distance=602, origin='ABE', destination='ATL'),
 Row(date=1061725, delay=69, distance=602, origin='ABE', destination='ATL'),
 Row(date=1061230, delay=0, distance=369, origin='ABE', destination='DTW'),
 Ro

In [ ]:
flightsDF.createOrReplaceTempView("flightsDF")

Create DF from RDD:

In [ ]:
sample_data = sc.parallelize([
(1, 'MacBook Pro', 2015, '15"', '16GB', '512GB SSD'
, 13.75, 9.48, 0.61, 4.02)
, (2, 'MacBook', 2016, '12"', '8GB', '256GB SSD'
, 11.04, 7.74, 0.52, 2.03)
, (3, 'MacBook Air', 2016, '13.3"', '8GB', '128GB SSD'
, 12.8, 8.94, 0.68, 2.96)
, (4, 'iMac', 2017, '27"', '64GB', '1TB SSD'
, 25.6, 8.0, 20.3, 20.8)
])

In [ ]:
sample_data.take(1)

[(1, 'MacBook Pro', 2015, '15"', '16GB', '512GB SSD', 13.75, 9.48, 0.61, 4.02)]

In [ ]:
sample_df_from_RDD = spark.createDataFrame(
sample_data
, [
'Id', 'Model', 'Year', 'ScreenSize', 'RAM', 'HDD', 'W', 'D', 'H', 'Weight'
]
)

In [ ]:
sample_df_from_RDD.take(2)

[Row(Id=1, Model='MacBook Pro', Year=2015, ScreenSize='15"', RAM='16GB', HDD='512GB SSD', W=13.75, D=9.48, H=0.61, Weight=4.02),
 Row(Id=2, Model='MacBook', Year=2016, ScreenSize='12"', RAM='8GB', HDD='256GB SSD', W=11.04, D=7.74, H=0.52, Weight=2.03)]

In [ ]:
sample_df_from_RDD.show()

+---+-----------+----+----------+----+---------+-----+----+----+------+
| Id|      Model|Year|ScreenSize| RAM|      HDD|    W|   D|   H|Weight|
+---+-----------+----+----------+----+---------+-----+----+----+------+
|  1|MacBook Pro|2015|       15"|16GB|512GB SSD|13.75|9.48|0.61|  4.02|
|  2|    MacBook|2016|       12"| 8GB|256GB SSD|11.04|7.74|0.52|  2.03|
|  3|MacBook Air|2016|     13.3"| 8GB|128GB SSD| 12.8|8.94|0.68|  2.96|
|  4|       iMac|2017|       27"|64GB|  1TB SSD| 25.6| 8.0|20.3|  20.8|
+---+-----------+----+----------+----+---------+-----+----+----+------+



In [ ]:
sample_df_from_RDD.printSchema()

root
 |-- Id: long (nullable = true)
 |-- Model: string (nullable = true)
 |-- Year: long (nullable = true)
 |-- ScreenSize: string (nullable = true)
 |-- RAM: string (nullable = true)
 |-- HDD: string (nullable = true)
 |-- W: double (nullable = true)
 |-- D: double (nullable = true)
 |-- H: double (nullable = true)
 |-- Weight: double (nullable = true)



Spark Dataframe from json file:

In [ ]:
sample_data_json_df = (
                        spark.read.json('/content/drive/My Drive/Pyspark/data/sample.json'))

In [ ]:
sample_data_json_df.show()

+----+----+---------+-----------+----+-----------+-----+-------+-----+---+
|   D|   H|      HDD|      Model| RAM| ScreenSize|    W| Weight| Year| Id|
+----+----+---------+-----------+----+-----------+-----+-------+-----+---+
|9.48|0.61|512GB SSD|MacBook Pro|16GB|        15"|13.75|   4.02| 2015|  1|
|7.74|0.52|256GB SSD|    MacBook| 8GB|        12"|11.04|   2.03| 2016|  2|
|8.94|0.68|128GB SSD|MacBook Air| 8GB|      13.3"| 12.8|   2.96| 2016|  3|
| 8.0|20.3|  1TB SSD|       iMac|64GB|        27"| 25.6|   20.8| 2017|  4|
+----+----+---------+-----------+----+-----------+-----+-------+-----+---+



In [ ]:
sample_data_json_df.printSchema()

root
 |--  D: double (nullable = true)
 |--  H: double (nullable = true)
 |--  HDD: string (nullable = true)
 |--  Model: string (nullable = true)
 |--  RAM: string (nullable = true)
 |--  ScreenSize: string (nullable = true)
 |--  W: double (nullable = true)
 |--  Weight: double (nullable = true)
 |--  Year: long (nullable = true)
 |-- Id: long (nullable = true)



In [ ]:
sample_df_from_RDD.rdd.take(1)


[Row(Id=1, Model='MacBook Pro', Year=2015, ScreenSize='15"', RAM='16GB', HDD='512GB SSD', W=13.75, D=9.48, H=0.61, Weight=4.02)]

In [ ]:
import pyspark.sql as sql
import pyspark.sql.functions as f

# Print schema separately (for debugging)
sample_df_from_RDD.printSchema()

# Transform the sample data
sample_data_transformed = (
    sample_df_from_RDD.rdd
    .map(lambda row: sql.Row(
        **row.asDict(),
        HDD_size=row.HDD.split(' ')[0]
    ))
    .map(lambda row: sql.Row(
        **row.asDict(),
        HDD_type=row.HDD.split(' ')[1]
    ))
    .map(lambda row: sql.Row(
        **row.asDict(),
        Volume=row.H * row.D * row.W
    ))
    .toDF()
    .select(
        *sample_df_from_RDD.columns,  # Unpacking existing columns
        'HDD_size',
        'HDD_type',
        f.round(f.col('Volume')).alias('Volume_cuIn')
    )
)

# Show transformed data
sample_data_transformed.show()


root
 |-- Id: long (nullable = true)
 |-- Model: string (nullable = true)
 |-- Year: long (nullable = true)
 |-- ScreenSize: string (nullable = true)
 |-- RAM: string (nullable = true)
 |-- HDD: string (nullable = true)
 |-- W: double (nullable = true)
 |-- D: double (nullable = true)
 |-- H: double (nullable = true)
 |-- Weight: double (nullable = true)

+---+-----------+----+----------+----+---------+-----+----+----+------+--------+--------+-----------+
| Id|      Model|Year|ScreenSize| RAM|      HDD|    W|   D|   H|Weight|HDD_size|HDD_type|Volume_cuIn|
+---+-----------+----+----------+----+---------+-----+----+----+------+--------+--------+-----------+
|  1|MacBook Pro|2015|       15"|16GB|512GB SSD|13.75|9.48|0.61|  4.02|   512GB|     SSD|       80.0|
|  2|    MacBook|2016|       12"| 8GB|256GB SSD|11.04|7.74|0.52|  2.03|   256GB|     SSD|       44.0|
|  3|MacBook Air|2016|     13.3"| 8GB|128GB SSD| 12.8|8.94|0.68|  2.96|   128GB|     SSD|       78.0|
|  4|       iMac|2017|       2

### **Explanation of the Code:**
This PySpark script processes a DataFrame `sample_df_from_RDD`, extracting and transforming specific columns before converting it back to a DataFrame.

---

## **1. Import Required Libraries**
```python
import pyspark.sql as sql
import pyspark.sql.functions as f
```
- `pyspark.sql` is imported to handle structured data using the Spark SQL module.
- `pyspark.sql.functions` (aliased as `f`) provides functions for column operations like `round()`.

---

## **2. Print Schema for Debugging**
```python
sample_df_from_RDD.printSchema()
```
- This **prints the schema** (column names and data types) of `sample_df_from_RDD` before applying transformations.
- It helps verify that the DataFrame is structured correctly.

---

## **3. Transform Data Using RDD Operations**
The dataset is **converted into an RDD** (Resilient Distributed Dataset) and modified before being converted back to a DataFrame.

### **a) Convert DataFrame to RDD**
```python
sample_df_from_RDD.rdd
```
- Converts `sample_df_from_RDD` (a Spark DataFrame) into an RDD of `Row` objects for row-wise transformations.

---

### **b) Extract `HDD_size`**
```python
.map(lambda row: sql.Row(
    **row.asDict(),
    HDD_size=row.HDD.split(' ')[0]  # Extracts the first part of the HDD column
))
```
- Each row is converted to a dictionary using `.asDict()`, then back to a `Row` object.
- The `HDD` column (e.g., `"500GB SSD"`) is split into words (`split(' ')`), and the **first word** (`500GB`) is stored as `HDD_size`.

---

### **c) Extract `HDD_type`**
```python
.map(lambda row: sql.Row(
    **row.asDict(),
    HDD_type=row.HDD.split(' ')[1]  # Extracts the second part of the HDD column
))
```
- Extracts the **second word** (`SSD`) from the `HDD` column and stores it as `HDD_type`.

---

### **d) Compute `Volume`**
```python
.map(lambda row: sql.Row(
    **row.asDict(),
    Volume=row.H * row.D * row.W  # Calculate the volume using height, depth, and width
))
```
- Computes the **volume** of an object using `H × D × W` and stores it as `Volume`.

---

## **4. Convert RDD Back to DataFrame**
```python
.toDF()
```
- Converts the transformed RDD **back to a Spark DataFrame**.

---

## **5. Select and Format Columns**
```python
.select(
    *sample_df_from_RDD.columns,  # Keep all original columns
    'HDD_size',
    'HDD_type',
    f.round(f.col('Volume')).alias('Volume_cuIn')  # Round the Volume and rename column
)
```
- Keeps **all original columns** from `sample_df_from_RDD`.
- Adds `HDD_size`, `HDD_type`, and **rounded** `Volume` (renamed to `Volume_cuIn`).

---

## **6. Display the Transformed Data**
```python
sample_data_transformed.show()
```
- Displays the transformed DataFrame.

---




## Performance optimizations Techniques

In [ ]:
import pyspark.sql.functions as f
import pandas as pd
from scipy import stats


In [ ]:
big_df = spark.range(0, 1000000).withColumn('val', f.rand())
big_df.cache()

big_df.show(3)


+---+-------------------+
| id|                val|
+---+-------------------+
|  0| 0.9186205554998382|
|  1|0.19159021324850545|
|  2|0.42624747849388045|
+---+-------------------+
only showing top 3 rows



This PySpark script creates a large DataFrame, caches it in memory for performance optimization, and then displays a few rows.

---

### **1. Create a Large DataFrame**  
```python
big_df = spark.range(0, 1000000).withColumn('val', f.rand())
```
- `spark.range(0, 1000000)`:  
  - Creates a **DataFrame** with a **single column (`id`)** containing values from `0` to `999999` (one million rows).  
- `.withColumn('val', f.rand())`:  
  - Adds a new column named **`val`**, where each row contains a **random number** between `0` and `1`.


### **2. Cache the DataFrame in Memory**  
```python
big_df.cache()
```
- **Stores the DataFrame in memory (or disk if needed)** for faster repeated access.  
- This prevents recomputation when `big_df` is used multiple times in subsequent operations.

💡 **Why use `cache()`?**  
- If `big_df` is used multiple times, Spark **won't re-evaluate** it.  
- Instead, it **fetches the precomputed result** from memory, improving performance.

---

### **3. Show the First 3 Rows**  
```python
big_df.show(3)

```
- Displays the **first 3 rows** of the DataFrame.


In [ ]:
@f.pandas_udf('double', f.PandasUDFType.SCALAR)
def pandas_pdf(v):

  return pd.Series(stats.norm.pdf(v))

big_df.withColumn('probability', pandas_pdf(big_df.val)).show(5)

/usr/local/lib/python3.11/dist-packages/pyspark/sql/pandas/functions.py:407: UserWarning: In Python 3.6+ and Spark 3.0+, it is preferred to specify type hints for pandas UDF instead of specifying pandas UDF type which will be deprecated in the future releases. See SPARK-28264 for more details.
  warnings.warn(


+---+-------------------+-------------------+
| id|                val|        probability|
+---+-------------------+-------------------+
|  0| 0.9186205554998382|0.26161785839371604|
|  1|0.19159021324850545| 0.3916871133600802|
|  2|0.42624747849388045| 0.3642983917289778|
|  3|0.18188943222960408|  0.392397306304047|
|  4| 0.8601473897522728|0.27558331033172945|
+---+-------------------+-------------------+
only showing top 5 rows



This code applies a **Pandas UDF** (User-Defined Function) to compute the **probability density function (PDF)** of a normal distribution for values in the **"val"** column of a Spark DataFrame.

---

#### **1️⃣ Define the Pandas UDF**
```python
@f.pandas_udf('double', f.PandasUDFType.SCALAR)
def pandas_pdf(v):
    return pd.Series(stats.norm.pdf(v))
```
- **`@f.pandas_udf('double', f.PandasUDFType.SCALAR)`**  
  - Defines a **Pandas UDF** that returns a column of `double` values.
  - `SCALAR`: This means the function **operates on each row (element-wise transformation)**.
- **Function `pandas_pdf(v)`**  
  - Takes a **Pandas Series `v`** (column values in batches).
  - Computes the **normal PDF** using `stats.norm.pdf(v)`.
  - Returns the computed values as a **Pandas Series**.

---

#### **2️⃣ Apply the UDF to the "val" Column**
```python
big_df.withColumn('probability', pandas_pdf(big_df.val)).show(5)
```
- `big_df.withColumn('probability', pandas_pdf(big_df.val))`:  
  - Creates a **new column "probability"** by applying `pandas_pdf` to the `"val"` column.
- `.show(5)`:  
  - Displays the first **5 rows** of the updated DataFrame.

---





The @f.pandas_udf decorator
preceding the method's declaration as this is key to registering a vectorized
UDF in PySpark and has only became available in Spark 2.3.

In [ ]:
#Performance Comparison

In [ ]:
# Pandas UDF Approach (Vectorized)
@f.pandas_udf('double', f.PandasUDFType.SCALAR)
def pandas_pdf(v):
    return pd.Series(stats.norm.pdf(v))

def test_pandas_pdf():
    return (big_df
        .withColumn('probability', pandas_pdf(big_df.val))  # Apply vectorized PDF
        .agg(f.count(f.col('probability')))  # Count non-null values
        .show()
    )

%timeit -n 1 test_pandas_pdf()


+------------------+
|count(probability)|
+------------------+
|           1000000|
+------------------+

+------------------+
|count(probability)|
+------------------+
|           1000000|
+------------------+

+------------------+
|count(probability)|
+------------------+
|           1000000|
+------------------+

+------------------+
|count(probability)|
+------------------+
|           1000000|
+------------------+

+------------------+
|count(probability)|
+------------------+
|           1000000|
+------------------+

+------------------+
|count(probability)|
+------------------+
|           1000000|
+------------------+

+------------------+
|count(probability)|
+------------------+
|           1000000|
+------------------+

2.35 s ± 1.2 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [ ]:
# Efficient because it processes entire batches of data at once.

In [ ]:
# Regular UDF Approach (Row-by-Row)
@f.udf('double')  # Standard PySpark UDF
def pdf(v):
    return float(stats.norm.pdf(v))  # Compute PDF per row

def test_pdf():
    return (big_df
        .withColumn('probability', pdf(big_df.val))  # Apply row-wise PDF
        .agg(f.count(f.col('probability')))  # Count non-null values
        .show()
    )

%timeit -n 1 test_pdf()


+------------------+
|count(probability)|
+------------------+
|           1000000|
+------------------+

+------------------+
|count(probability)|
+------------------+
|           1000000|
+------------------+

+------------------+
|count(probability)|
+------------------+
|           1000000|
+------------------+

+------------------+
|count(probability)|
+------------------+
|           1000000|
+------------------+

+------------------+
|count(probability)|
+------------------+
|           1000000|
+------------------+

+------------------+
|count(probability)|
+------------------+
|           1000000|
+------------------+

+------------------+
|count(probability)|
+------------------+
|           1000000|
+------------------+

1min 53s ± 1.85 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [ ]:
# Slower because it processes each row individually, causing frequent Python-JVM conversions.

## Infering the schema With reflection


In [ ]:
sample_rdd = sc.textFile('/content/drive/My Drive/Pyspark/data/sample.csv')
sample_rdd.take(5)

['Id, Model, Year, ScreenSize, RAM, HDD, W, D, H, Weight',
 '1,MacBook Pro,2015,"15\\"",16GB,512GB SSD,13.75,9.48,0.61,4.02',
 '2,MacBook,2016,"12\\"",8GB,256GB SSD,11.04,7.74,0.52,2.03',
 '3,MacBook Air,2016,"13.3\\"",8GB,128GB SSD,12.8,8.94,0.68,2.96',
 '4,iMac,2017,"27\\"",64GB,1TB SSD,25.6,8.0,20.3,20.8']

In [ ]:
# sample_data_json_df = (
#                         spark.read.json('/content/drive/My Drive/Pyspark/data/sample.json'))

In [ ]:
import pyspark.sql as sql

header = sample_rdd.first()

sample_data_rdd_row = (
    sample_rdd
    .filter(lambda row: row != header)
    .map(lambda row: row.split(','))
    .map(lambda row:
        sql.Row(
            Id=int(row[0])
            , Model=row[1]
            , Year=int(row[2])
            , ScreenSize=row[3]
            , RAM=row[4]
            , HDD=row[5]
            , W=float(row[6])
            , D=float(row[7])
            , H=float(row[8])
            , Weight=float(row[9])
        )
    )
)

In [ ]:
spark.createDataFrame(sample_data_rdd_row).show(5)


+---+-----------+----+----------+----+---------+-----+----+----+------+
| Id|      Model|Year|ScreenSize| RAM|      HDD|    W|   D|   H|Weight|
+---+-----------+----+----------+----+---------+-----+----+----+------+
|  1|MacBook Pro|2015|    "15\""|16GB|512GB SSD|13.75|9.48|0.61|  4.02|
|  2|    MacBook|2016|    "12\""| 8GB|256GB SSD|11.04|7.74|0.52|  2.03|
|  3|MacBook Air|2016|  "13.3\""| 8GB|128GB SSD| 12.8|8.94|0.68|  2.96|
|  4|       iMac|2017|    "27\""|64GB|  1TB SSD| 25.6| 8.0|20.3|  20.8|
+---+-----------+----+----------+----+---------+-----+----+----+------+



## Defining the schema dynamically in code.

In [ ]:
import pyspark.sql.types as typ

sch = typ.StructType([
      typ.StructField('Id', typ.LongType(), False)
    , typ.StructField('Model', typ.StringType(), True)
    , typ.StructField('Year', typ.IntegerType(), True)
    , typ.StructField('ScreenSize', typ.StringType(), True)
    , typ.StructField('RAM', typ.StringType(), True)
    , typ.StructField('HDD', typ.StringType(), True)
    , typ.StructField('W', typ.DoubleType(), True)
    , typ.StructField('D', typ.DoubleType(), True)
    , typ.StructField('H', typ.DoubleType(), True)
    , typ.StructField('Weight', typ.DoubleType(), True)
])

This code defines a schema for a PySpark DataFrame using `StructType` and `StructField` from `pyspark.sql.types`.  

- `StructType([])`: Represents the overall schema as a collection of fields.  
- `StructField(name, dataType, nullable)`: Defines each column with its name, data type, and whether it can contain `NULL` values.  

### Schema Details:  
| Column Name  | Data Type       | Nullable | Description  |
|-------------|----------------|----------|-------------|
| `Id`        | `LongType`      | `False`  | Unique identifier for each row (mandatory). |
| `Model`     | `StringType`    | `True`   | Name of the model. |
| `Year`      | `IntegerType`   | `True`   | Manufacturing year. |
| `ScreenSize`| `StringType`    | `True`   | Screen size of the device. |
| `RAM`       | `StringType`    | `True`   | RAM size of the device. |
| `HDD`       | `StringType`    | `True`   | Hard drive description. |
| `W`         | `DoubleType`    | `True`   | Width of the device. |
| `D`         | `DoubleType`    | `True`   | Depth of the device. |
| `H`         | `DoubleType`    | `True`   | Height of the device. |
| `Weight`    | `DoubleType`    | `True`   | Weight of the device. |

In [ ]:
sample_rdd = sc.textFile('/content/drive/My Drive/Pyspark/data/sample.csv')

header = sample_rdd.first()

sample_rdd = (
    sample_rdd
    .filter(lambda row: row != header)
    .map(lambda row: row.split(','))
    .map(lambda row: (
                int(row[0])
                , row[1]
                , int(row[2])
                , row[3]
                , row[4]
                , row[5]
                , float(row[6])
                , float(row[7])
                , float(row[8])
                , float(row[9])
        )
    )
)


In [ ]:
sample_data_schema = spark.createDataFrame(sample_rdd, schema=sch)
sample_data_schema.show()

+---+-----------+----+----------+----+---------+-----+----+----+------+
| Id|      Model|Year|ScreenSize| RAM|      HDD|    W|   D|   H|Weight|
+---+-----------+----+----------+----+---------+-----+----+----+------+
|  1|MacBook Pro|2015|    "15\""|16GB|512GB SSD|13.75|9.48|0.61|  4.02|
|  2|    MacBook|2016|    "12\""| 8GB|256GB SSD|11.04|7.74|0.52|  2.03|
|  3|MacBook Air|2016|  "13.3\""| 8GB|128GB SSD| 12.8|8.94|0.68|  2.96|
|  4|       iMac|2017|    "27\""|64GB|  1TB SSD| 25.6| 8.0|20.3|  20.8|
+---+-----------+----+----------+----+---------+-----+----+----+------+



## Temporary Table

In [ ]:
# Create a temporary view
sample_data_schema.createTempView('sample_data_view')

In [ ]:
# We will start writing sql code using this view


In [ ]:
spark.sql('''
    SELECT Model
        , Year
        , RAM
        , HDD
    FROM sample_data_view
''').show()

+-----------+----+----+---------+
|      Model|Year| RAM|      HDD|
+-----------+----+----+---------+
|MacBook Pro|2015|16GB|512GB SSD|
|    MacBook|2016| 8GB|256GB SSD|
|MacBook Air|2016| 8GB|128GB SSD|
|       iMac|2017|64GB|  1TB SSD|
+-----------+----+----+---------+



In [ ]:
sample_data_schema.createOrReplaceTempView('sample_data_view')

In [ ]:
spark.sql('''
    SELECT *
    FROM sample_data_view
''').show()

+---+-----------+----+----------+----+---------+-----+----+----+------+
| Id|      Model|Year|ScreenSize| RAM|      HDD|    W|   D|   H|Weight|
+---+-----------+----+----------+----+---------+-----+----+----+------+
|  1|MacBook Pro|2015|    "15\""|16GB|512GB SSD|13.75|9.48|0.61|  4.02|
|  2|    MacBook|2016|    "12\""| 8GB|256GB SSD|11.04|7.74|0.52|  2.03|
|  3|MacBook Air|2016|  "13.3\""| 8GB|128GB SSD| 12.8|8.94|0.68|  2.96|
|  4|       iMac|2017|    "27\""|64GB|  1TB SSD| 25.6| 8.0|20.3|  20.8|
+---+-----------+----+----------+----+---------+-----+----+----+------+



In [ ]:
spark.sql('''
    SELECT Model
      , Year
      , RAM
      , HDD
      , ScreenSize
    FROM sample_data_view
''').show()

+-----------+----+----+---------+----------+
|      Model|Year| RAM|      HDD|ScreenSize|
+-----------+----+----+---------+----------+
|MacBook Pro|2015|16GB|512GB SSD|    "15\""|
|    MacBook|2016| 8GB|256GB SSD|    "12\""|
|MacBook Air|2016| 8GB|128GB SSD|  "13.3\""|
|       iMac|2017|64GB|  1TB SSD|    "27\""|
+-----------+----+----+---------+----------+



## Let's play with the dataframe using sql commands


In [ ]:
df = sc.parallelize([
      ('MacBook Pro', 'Laptop')
    , ('MacBook', 'Laptop')
    , ('MacBook Air', 'Laptop')
    , ('iMac', 'Desktop')
]).toDF(['Model', 'FormFactor'])

df.createOrReplaceTempView('models')

In [ ]:
sample_data_schema.createOrReplaceTempView('sample_data_view')

In [ ]:
spark.sql('''
    SELECT a.*
        , b.FormFactor
    FROM sample_data_view AS a
    LEFT JOIN models AS b
        ON a.Model == b.Model
    ORDER BY Weight DESC
''').show()

+---+-----------+----+----------+----+---------+-----+----+----+------+----------+
| Id|      Model|Year|ScreenSize| RAM|      HDD|    W|   D|   H|Weight|FormFactor|
+---+-----------+----+----------+----+---------+-----+----+----+------+----------+
|  4|       iMac|2017|    "27\""|64GB|  1TB SSD| 25.6| 8.0|20.3|  20.8|   Desktop|
|  1|MacBook Pro|2015|    "15\""|16GB|512GB SSD|13.75|9.48|0.61|  4.02|    Laptop|
|  3|MacBook Air|2016|  "13.3\""| 8GB|128GB SSD| 12.8|8.94|0.68|  2.96|    Laptop|
|  2|    MacBook|2016|    "12\""| 8GB|256GB SSD|11.04|7.74|0.52|  2.03|    Laptop|
+---+-----------+----+----------+----+---------+-----+----+----+------+----------+



Werun an **SQL query in PySpark** using `spark.sql()` to join two tables and retrieve data in a structured way.


1. **Querying Tables**  
   - `sample_data_view (a)`: Main table containing sample data.  
   - `models (b)`: Lookup table that includes `FormFactor` information.  

2. **Join Operation (`LEFT JOIN`)**  
   - Matches rows from `sample_data_view` (`a`) with `models` (`b`) based on the `Model` column.  
   - Keeps all rows from `sample_data_view`, even if no match is found in `models` (null values will appear for `FormFactor` if there’s no match).  

3. **Column Selection (`a.*, b.FormFactor`)**  
   - Selects all columns from `sample_data_view` (`a.*`).  
   - Adds `FormFactor` from `models` (`b`).  

4. **Sorting (`ORDER BY Weight DESC`)**  
   - Orders results by the `Weight` column in **descending order** (heaviest first).  

5. **Displaying Results (`.show()`)**  
   - Shows the first few rows of the resulting DataFrame.  

In [ ]:
spark.sql('''
    SELECT b.FormFactor
        , COUNT(*) AS ComputerCnt
    FROM sample_data_view AS a
    LEFT JOIN models AS b
        ON a.Model == b.Model
    GROUP BY FormFactor
''').show()

+----------+-----------+
|FormFactor|ComputerCnt|
+----------+-----------+
|    Laptop|          3|
|   Desktop|          1|
+----------+-----------+



This SQL query **counts the number of computers** for each `FormFactor` by joining two datasets:  

- `sample_data_view` (**`a`**): Contains information about different computers.  
- `models` (**`b`**): Contains additional details, including the `FormFactor` for each model.  

---

1. **LEFT JOIN (`ON a.Model == b.Model`)**  
   - Combines data from both tables based on the `Model` column.  
   - Ensures all records from `sample_data_view` (`a`) remain, even if `models` (`b`) lacks a matching `Model`.  
   - If there's no match, `FormFactor` will be `NULL` for those rows.  

2. **COUNT(*) AS ComputerCnt**  
   - Counts the number of computers (rows) for each `FormFactor`.  
   - Groups results by `FormFactor`.  

3. **GROUP BY FormFactor**  
   - Aggregates results so that each unique `FormFactor` gets a count of associated computers.  

4. **.show()**  
   - Displays the first few rows of the resulting DataFrame.  


## DataFrame Transformation

.select() transformation:

In [ ]:
# select Model and ScreenSize from the DataFrame
sample_data_schema.select('Model', 'ScreenSize').show()

+-----------+----------+
|      Model|ScreenSize|
+-----------+----------+
|MacBook Pro|    "15\""|
|    MacBook|    "12\""|
|MacBook Air|  "13.3\""|
|       iMac|    "27\""|
+-----------+----------+



.filter() transformation

In [ ]:
# extract only machines from 2015 onwards
sample_data_schema.filter(sample_data_schema.Year > 2015).show()


+---+-----------+----+----------+----+---------+-----+----+----+------+
| Id|      Model|Year|ScreenSize| RAM|      HDD|    W|   D|   H|Weight|
+---+-----------+----+----------+----+---------+-----+----+----+------+
|  2|    MacBook|2016|    "12\""| 8GB|256GB SSD|11.04|7.74|0.52|  2.03|
|  3|MacBook Air|2016|  "13.3\""| 8GB|128GB SSD| 12.8|8.94|0.68|  2.96|
|  4|       iMac|2017|    "27\""|64GB|  1TB SSD| 25.6| 8.0|20.3|  20.8|
+---+-----------+----+----------+----+---------+-----+----+----+------+



.groupBy() transformation

In [ ]:
sample_data_schema.groupBy('RAM').count().show()

+----+-----+
| RAM|count|
+----+-----+
|16GB|    1|
| 8GB|    2|
|64GB|    1|
+----+-----+



.orderBy() transformation

In [ ]:
# sort by width (W)
sample_data_schema.orderBy('W').show()

+---+-----------+----+----------+----+---------+-----+----+----+------+
| Id|      Model|Year|ScreenSize| RAM|      HDD|    W|   D|   H|Weight|
+---+-----------+----+----------+----+---------+-----+----+----+------+
|  2|    MacBook|2016|    "12\""| 8GB|256GB SSD|11.04|7.74|0.52|  2.03|
|  3|MacBook Air|2016|  "13.3\""| 8GB|128GB SSD| 12.8|8.94|0.68|  2.96|
|  1|MacBook Pro|2015|    "15\""|16GB|512GB SSD|13.75|9.48|0.61|  4.02|
|  4|       iMac|2017|    "27\""|64GB|  1TB SSD| 25.6| 8.0|20.3|  20.8|
+---+-----------+----+----------+----+---------+-----+----+----+------+



In [ ]:
# sort by height (H) in descending order
sample_data_schema.orderBy(f.col('H').desc()).show()

+---+-----------+----+----------+----+---------+-----+----+----+------+
| Id|      Model|Year|ScreenSize| RAM|      HDD|    W|   D|   H|Weight|
+---+-----------+----+----------+----+---------+-----+----+----+------+
|  4|       iMac|2017|    "27\""|64GB|  1TB SSD| 25.6| 8.0|20.3|  20.8|
|  3|MacBook Air|2016|  "13.3\""| 8GB|128GB SSD| 12.8|8.94|0.68|  2.96|
|  1|MacBook Pro|2015|    "15\""|16GB|512GB SSD|13.75|9.48|0.61|  4.02|
|  2|    MacBook|2016|    "12\""| 8GB|256GB SSD|11.04|7.74|0.52|  2.03|
+---+-----------+----+----------+----+---------+-----+----+----+------+



.withColumn() transformation

In [ ]:
# split the HDD into size and type
sample_data_schema.withColumn('HDDSplit', f.split(f.col('HDD'), ' ')).show()

+---+-----------+----+----------+----+---------+-----+----+----+------+------------+
| Id|      Model|Year|ScreenSize| RAM|      HDD|    W|   D|   H|Weight|    HDDSplit|
+---+-----------+----+----------+----+---------+-----+----+----+------+------------+
|  1|MacBook Pro|2015|    "15\""|16GB|512GB SSD|13.75|9.48|0.61|  4.02|[512GB, SSD]|
|  2|    MacBook|2016|    "12\""| 8GB|256GB SSD|11.04|7.74|0.52|  2.03|[256GB, SSD]|
|  3|MacBook Air|2016|  "13.3\""| 8GB|128GB SSD| 12.8|8.94|0.68|  2.96|[128GB, SSD]|
|  4|       iMac|2017|    "27\""|64GB|  1TB SSD| 25.6| 8.0|20.3|  20.8|  [1TB, SSD]|
+---+-----------+----+----------+----+---------+-----+----+----+------+------------+



In [ ]:
# do the same as withColumn
(sample_data_schema.select(f.col('*')
, f.split(f.col('HDD'), ' ').alias('HDD_Array')).show())

+---+-----------+----+----------+----+---------+-----+----+----+------+------------+
| Id|      Model|Year|ScreenSize| RAM|      HDD|    W|   D|   H|Weight|   HDD_Array|
+---+-----------+----+----------+----+---------+-----+----+----+------+------------+
|  1|MacBook Pro|2015|    "15\""|16GB|512GB SSD|13.75|9.48|0.61|  4.02|[512GB, SSD]|
|  2|    MacBook|2016|    "12\""| 8GB|256GB SSD|11.04|7.74|0.52|  2.03|[256GB, SSD]|
|  3|MacBook Air|2016|  "13.3\""| 8GB|128GB SSD| 12.8|8.94|0.68|  2.96|[128GB, SSD]|
|  4|       iMac|2017|    "27\""|64GB|  1TB SSD| 25.6| 8.0|20.3|  20.8|  [1TB, SSD]|
+---+-----------+----+----------+----+---------+-----+----+----+------+------------+



.join() transformation

In [ ]:
models_df = sc.parallelize([
      ('MacBook Pro', 'Laptop')
    , ('MacBook Air', 'Laptop')
    , ('iMac', 'Desktop')
]).toDF(['Model', 'FormFactor'])

(
    sample_data_schema
    .join(
        models_df # Join with models_df
        , sample_data_schema.Model == models_df.Model # Match on 'Model'
        , 'left' # Perform LEFT JOIN (Keep all rows from sample_data_schema)
    ).show()
)

+---+-----------+----+----------+----+---------+-----+----+----+------+-----------+----------+
| Id|      Model|Year|ScreenSize| RAM|      HDD|    W|   D|   H|Weight|      Model|FormFactor|
+---+-----------+----+----------+----+---------+-----+----+----+------+-----------+----------+
|  2|    MacBook|2016|    "12\""| 8GB|256GB SSD|11.04|7.74|0.52|  2.03|       NULL|      NULL|
|  1|MacBook Pro|2015|    "15\""|16GB|512GB SSD|13.75|9.48|0.61|  4.02|MacBook Pro|    Laptop|
|  3|MacBook Air|2016|  "13.3\""| 8GB|128GB SSD| 12.8|8.94|0.68|  2.96|MacBook Air|    Laptop|
|  4|       iMac|2017|    "27\""|64GB|  1TB SSD| 25.6| 8.0|20.3|  20.8|       iMac|   Desktop|
+---+-----------+----+----------+----+---------+-----+----+----+------+-----------+----------+



In [ ]:
sample_data_schema.join(
        models_df
        , sample_data_schema.Model == models_df.Model
        , 'right'
    ).show()

+---+-----------+----+----------+----+---------+-----+----+----+------+-----------+----------+
| Id|      Model|Year|ScreenSize| RAM|      HDD|    W|   D|   H|Weight|      Model|FormFactor|
+---+-----------+----+----------+----+---------+-----+----+----+------+-----------+----------+
|  1|MacBook Pro|2015|    "15\""|16GB|512GB SSD|13.75|9.48|0.61|  4.02|MacBook Pro|    Laptop|
|  3|MacBook Air|2016|  "13.3\""| 8GB|128GB SSD| 12.8|8.94|0.68|  2.96|MacBook Air|    Laptop|
|  4|       iMac|2017|    "27\""|64GB|  1TB SSD| 25.6| 8.0|20.3|  20.8|       iMac|   Desktop|
+---+-----------+----+----------+----+---------+-----+----+----+------+-----------+----------+



The SEMI and ANTI joins are somewhat recent additions. The SEMI join keeps all
the records from the left DataFrame that are matched with the records in the
right DataFrame (as with the RIGHT join) but only keeps the columns from the
left DataFrame; the ANTI join is the opposite of the SEMI join—it keeps only
the records that are not found in the right DataFrame.

In [ ]:
sample_data_schema.join(
        models_df
        , sample_data_schema.Model == models_df.Model
        , 'left_anti'
    ).show()

+---+-------+----+----------+---+---------+-----+----+----+------+
| Id|  Model|Year|ScreenSize|RAM|      HDD|    W|   D|   H|Weight|
+---+-------+----+----------+---+---------+-----+----+----+------+
|  2|MacBook|2016|    "12\""|8GB|256GB SSD|11.04|7.74|0.52|  2.03|
+---+-------+----+----------+---+---------+-----+----+----+------+



In [ ]:
sample_data_schema.join(
        models_df
        , sample_data_schema.Model == models_df.Model
        , 'left_semi'
    ).show()

+---+-----------+----+----------+----+---------+-----+----+----+------+
| Id|      Model|Year|ScreenSize| RAM|      HDD|    W|   D|   H|Weight|
+---+-----------+----+----------+----+---------+-----+----+----+------+
|  3|MacBook Air|2016|  "13.3\""| 8GB|128GB SSD| 12.8|8.94|0.68|  2.96|
|  1|MacBook Pro|2015|    "15\""|16GB|512GB SSD|13.75|9.48|0.61|  4.02|
|  4|       iMac|2017|    "27\""|64GB|  1TB SSD| 25.6| 8.0|20.3|  20.8|
+---+-----------+----+----------+----+---------+-----+----+----+------+



.unionAll() transformation

In [ ]:
another_macBookPro = sc.parallelize([
                          (5, 'MacBook Pro', 2018, '15"', '16GB', '256GB SSD', 13.75, 9.48, 0.61, 4.02)])\
                          .toDF(sample_data_schema.columns)
sample_data_schema.unionAll(another_macBookPro).show()

+---+-----------+----+----------+----+---------+-----+----+----+------+
| Id|      Model|Year|ScreenSize| RAM|      HDD|    W|   D|   H|Weight|
+---+-----------+----+----------+----+---------+-----+----+----+------+
|  1|MacBook Pro|2015|    "15\""|16GB|512GB SSD|13.75|9.48|0.61|  4.02|
|  2|    MacBook|2016|    "12\""| 8GB|256GB SSD|11.04|7.74|0.52|  2.03|
|  3|MacBook Air|2016|  "13.3\""| 8GB|128GB SSD| 12.8|8.94|0.68|  2.96|
|  4|       iMac|2017|    "27\""|64GB|  1TB SSD| 25.6| 8.0|20.3|  20.8|
|  5|MacBook Pro|2018|       15"|16GB|256GB SSD|13.75|9.48|0.61|  4.02|
+---+-----------+----+----------+----+---------+-----+----+----+------+



.distinct() transformation:

In [ ]:
# select the distinct values from the RAM column
sample_data_schema.select('RAM').distinct().show()

+----+
| RAM|
+----+
|16GB|
| 8GB|
|64GB|
+----+



.repartition() transformation:

In [ ]:
sample_data_schema_rep = (sample_data_schema.repartition(2, 'Year'))
sample_data_schema_rep.rdd.getNumPartitions()

2

.fillna() transformation

In [ ]:
missing_df = sc.parallelize([
                              (None, 36.3, 24.2)
                              , (1.6, 32.1, 27.9)
                              , (3.2, 38.7, 24.7)
                              , (2.8, None, 23.9)
                              , (3.9, 34.1, 27.9)
                              , (9.2, None, None)
                              ]).toDF(['A', 'B', 'C'])

missing_df.show()

+----+----+----+
|   A|   B|   C|
+----+----+----+
|NULL|36.3|24.2|
| 1.6|32.1|27.9|
| 3.2|38.7|24.7|
| 2.8|NULL|23.9|
| 3.9|34.1|27.9|
| 9.2|NULL|NULL|
+----+----+----+



In [ ]:
#Replace missing value with 21.4
missing_df.fillna(21.4).show()

+----+----+----+
|   A|   B|   C|
+----+----+----+
|21.4|36.3|24.2|
| 1.6|32.1|27.9|
| 3.2|38.7|24.7|
| 2.8|21.4|23.9|
| 3.9|34.1|27.9|
| 9.2|21.4|21.4|
+----+----+----+



In [ ]:
miss_dict = (missing_df.agg(f.mean('A').alias('A')
      , f.mean('B').alias('B')
      , f.mean('C').alias('C')
      )
).toPandas().to_dict('records')[0]

missing_df.fillna(miss_dict).show()

+----+------------------+-----+
|   A|                 B|    C|
+----+------------------+-----+
|4.14|              36.3| 24.2|
| 1.6|              32.1| 27.9|
| 3.2|              38.7| 24.7|
| 2.8|35.300000000000004| 23.9|
| 3.9|              34.1| 27.9|
| 9.2|35.300000000000004|25.72|
+----+------------------+-----+



.dropna() transformation

In [ ]:
#Remove missing value
missing_df.dropna().show()

+---+----+----+
|  A|   B|   C|
+---+----+----+
|1.6|32.1|27.9|
|3.2|38.7|24.7|
|3.9|34.1|27.9|
+---+----+----+



In [ ]:
missing_df.dropna(thresh=2).show()

+----+----+----+
|   A|   B|   C|
+----+----+----+
|NULL|36.3|24.2|
| 1.6|32.1|27.9|
| 3.2|38.7|24.7|
| 2.8|NULL|23.9|
| 3.9|34.1|27.9|
+----+----+----+



### **Understanding `dropna(thresh=2)` in PySpark**

```python
missing_df.dropna(thresh=2).show()
```

---

### **What This Code Does**
1. **`dropna(thresh=2)`**  
   - Removes rows that have **less than 2 non-null (non-missing) values**.
   - If a row has **at least 2 non-null values**, it remains in the DataFrame.
   - If a row has **fewer than 2 non-null values**, it gets dropped.

2. **`.show()`**  
   - Displays the resulting DataFrame after applying `dropna()`.

---

### **Example Data Before Applying `dropna(thresh=2)`**
| ID | Name  | Age  | Salary  |
|----|-------|------|--------|
| 1  | Alice | 25   | 50000  |
| 2  | Bob   | None | None   |
| 3  | None  | None | 30000  |
| 4  | John  | 30   | None   |

---

### **After Applying `dropna(thresh=2)`**
| ID | Name  | Age  | Salary  |
|----|-------|------|--------|
| 1  | Alice | 25   | 50000  |
| 4  | John  | 30   | None   |

- **Row 2** (`Bob, None, None`) was removed (only 1 non-null value: `ID`).
- **Row 3** (`None, None, 30000`) was removed (only 1 non-null value: `Salary`).
- **Rows 1 and 4** remained because they had **at least 2 non-null values**.

---


.dropDuplicates()transformation:

In [ ]:
# The .dropDuplicates(...) transformation, as the name suggests, removes
# duplicated records.

In [ ]:
dupes_df = sc.parallelize([
        (1.6, 32.1, 27.9)
        , (3.2, 38.7, 24.7)
        , (3.9, 34.1, 27.9)
        , (3.2, 38.7, 24.7)
        ]).toDF(['A', 'B', 'C'])
dupes_df.dropDuplicates().show()

+---+----+----+
|  A|   B|   C|
+---+----+----+
|1.6|32.1|27.9|
|3.2|38.7|24.7|
|3.9|34.1|27.9|
+---+----+----+



.summary() and .describe() transformations :

In [ ]:
# The .summary() and .describe() transformations produce similar descriptive
# statistics, with the .summary() transformation additionally producing quartiles.

In [ ]:
sample_data_schema.select('W').summary().show()

+-------+------------------+
|summary|                 W|
+-------+------------------+
|  count|                 4|
|   mean|15.797500000000001|
| stddev| 6.630738395281983|
|    min|             11.04|
|    25%|             11.04|
|    50%|              12.8|
|    75%|             13.75|
|    max|              25.6|
+-------+------------------+



In [ ]:
sample_data_schema.select('W').describe().show()

+-------+------------------+
|summary|                 W|
+-------+------------------+
|  count|                 4|
|   mean|15.797500000000001|
| stddev| 6.630738395281983|
|    min|             11.04|
|    max|              25.6|
+-------+------------------+



In [ ]:
sample_data_schema.summary().show()

+-------+------------------+-------+-----------------+----------+----+---------+------------------+------------------+-----------------+-----------------+
|summary|                Id|  Model|             Year|ScreenSize| RAM|      HDD|                 W|                 D|                H|           Weight|
+-------+------------------+-------+-----------------+----------+----+---------+------------------+------------------+-----------------+-----------------+
|  count|                 4|      4|                4|         4|   4|        4|                 4|                 4|                4|                4|
|   mean|               2.5|   NULL|           2016.0|      NULL|NULL|     NULL|15.797500000000001|              8.54|           5.5275|7.452500000000001|
| stddev|1.2909944487358056|   NULL|0.816496580927726|      NULL|NULL|     NULL| 6.630738395281983|0.8114185110040317|9.848551077865888|8.935395439859764|
|    min|                 1|MacBook|             2015|    "12\""|16GB|

.freqItems() transformation:

The .freqItems(...) transformation returns a list of frequent items from a
column. You can also specify a minSupport parameter that will throw away
items that are below a certain threshold.

In [ ]:
sample_data_schema.freqItems(['RAM']).show()

+-----------------+
|    RAM_freqItems|
+-----------------+
|[16GB, 64GB, 8GB]|
+-----------------+



## Spark DataFrame Actions


.show() action:

In [ ]:
sample_data_schema.select('W').describe().show()

+-------+------------------+
|summary|                 W|
+-------+------------------+
|  count|                 4|
|   mean|15.797500000000001|
| stddev| 6.630738395281983|
|    min|             11.04|
|    max|              25.6|
+-------+------------------+



.collect() action:


In [ ]:
sample_data_schema.groupBy('Year').count().collect()

[Row(Year=2015, count=1), Row(Year=2016, count=2), Row(Year=2017, count=1)]

.take() action:

In [ ]:
sample_data_schema.take(2)

[Row(Id=1, Model='MacBook Pro', Year=2015, ScreenSize='"15\\""', RAM='16GB', HDD='512GB SSD', W=13.75, D=9.48, H=0.61, Weight=4.02),
 Row(Id=2, Model='MacBook', Year=2016, ScreenSize='"12\\""', RAM='8GB', HDD='256GB SSD', W=11.04, D=7.74, H=0.52, Weight=2.03)]

.toPandas() action:


In [ ]:
# The .toPandas() action, as the name suggests, converts the Spark DataFrame
# into a pandas DataFrame.

In [ ]:
sample_data_schema.toPandas()

,Id,Model,Year,ScreenSize,RAM,HDD,W,D,H,Weight
0,1,MacBook Pro,2015,"""15\""""",16GB,512GB SSD,13.75,9.48,0.61,4.02
1,2,MacBook,2016,"""12\""""",8GB,256GB SSD,11.04,7.74,0.52,2.03
2,3,MacBook Air,2016,"""13.3\""""",8GB,128GB SSD,12.80,8.94,0.68,2.96
3,4,iMac,2017,"""27\""""",64GB,1TB SSD,25.60,8.00,20.30,20.80


from matplotlib import pyplot as plt
_df_0['Id'].plot(kind='hist', bins=20, title='Id')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_1['Year'].plot(kind='hist', bins=20, title='Year')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_2['W'].plot(kind='hist', bins=20, title='W')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_3['D'].plot(kind='hist', bins=20, title='D')
plt.gca().spines[['top', 'right',]].set_visible(False)

...

from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(_df_24['ScreenSize'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(_df_24, x='Id', y='ScreenSize', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

<string>:5: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.



from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(_df_25['RAM'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(_df_25, x='Id', y='RAM', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

<string>:5: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.



from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(_df_26['HDD'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(_df_26, x='Id', y='HDD', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)